In [1]:
from glob import glob
from PIL import Image
from tqdm import tqdm
import os
import cv2
import json
import shutil
import base64
import numpy as np

In [2]:
root_path = "/home/ubuntu/workspace/seunghoon/z_dataset"
target_folder_name = "diablo2_dataset_ver2"

src_folder_name = "labelme_dataset_processing2"
dst_folder_name = "labelme_dataset_processing3"

In [3]:
src_imgs = glob(os.path.join(root_path, target_folder_name, src_folder_name, "*"))
src_imgs = [i for i in src_imgs if ".png" in i or ".jpg" in i and not ".json" in i]
print(len(src_imgs))

22886


In [ ]:
save_dir_path = os.path.join(root_path, target_folder_name, dst_folder_name)
cnt = 0
folder_cnt = 0
folder_num = str(folder_cnt).zfill(3)
for img in tqdm(src_imgs):  
    if cnt % 100 == 0:
        folder_cnt += 1
        folder_num = str(folder_cnt).zfill(3)
        
    # labelme_dict 생성
    PIL_img = Image.open(img)
    width, height = PIL_img.size
    
    cv_img = cv2.imread(img)
    _, img_encoded = cv2.imencode('.jpg', cv_img)
    jpg_as_text = base64.b64encode(img_encoded).decode()
    
    labelme_dict = {
        'version': '5.1.1',
        'flags': {},
        'shapes': [],
        'imagePath': f"C:/{dst_folder_name}/{folder_num}/{os.path.basename(img)}",
        'imageData': jpg_as_text,
        'imageHeight': height,
        'imageWidth': width
    }
    
    # naver_ocr로 생성된 json에서 필요한 정보만 추출해서 labelme_dict에 추가
    json_path = img.replace(os.path.splitext(img)[1], ".json")
    with open(json_path, "r")as f:
        data = json.load(f)
        
    for _, v in data.items():
        final_text = v[0]
        coors = v[1]

        left_up = coors[0]
        right_up = coors[1]
        right_down = coors[2]
        left_down = coors[3]
        
        shape = {
            'label': final_text,
            'points': [left_up, right_up, right_down, left_down],
            'group_id': None,
            'shape_type': 'polygon',
            'flags': {}
        }
        labelme_dict['shapes'].append(shape)
        
    # 이미지와 labelme_dict로 만들어진 json을 1폴더에 100개씩 저장
    os.makedirs(os.path.join(save_dir_path, folder_num), exist_ok=True)
    
    with open(os.path.join(save_dir_path, folder_num, f"{os.path.splitext(os.path.basename(img))[0]}.json"), "w") as f:
        json.dump(labelme_dict, f, ensure_ascii=False, indent = "\n")
        
    shutil.copy(img, os.path.join(save_dir_path, folder_num, f"{os.path.basename(img)}"))
        
    cnt += 1

 19%|█▉        | 4411/22886 [02:07<06:05, 50.56it/s]